In [1]:
import torch
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy

import torch.nn.init as init

print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())


PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Convert to tensor and scale to [0, 1]
ts = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0,), (1,))])
mnist_trainset = datasets.MNIST('/home/c/cl237/', train=True, download=True, transform=ts)
mnist_testset = datasets.MNIST(root='/home/c/cl237/', train=False, download=True, transform=ts)

In [3]:
from torch.utils.data import Subset
train_subset_size = int(0.1 * len(mnist_trainset))
test_subset_size = int(0.0999 * len(mnist_testset))
train_indices = list(range(len(mnist_trainset)))
test_indices = list(range(len(mnist_testset)))
torch.manual_seed(10)

seed_value = 10
np.random.seed(seed_value)

np.random.shuffle(train_indices)
np.random.shuffle(test_indices)

train_subset_indices = train_indices[:train_subset_size]
test_subset_indices = test_indices[:test_subset_size]

mnist_trainset = Subset(mnist_trainset, train_subset_indices)
mnist_testset = Subset(mnist_testset, test_subset_indices)

In [4]:
x_d0 = mnist_trainset[0][0].size()[0]
x_d1 = mnist_trainset[0][0].size()[1]
x_d2 = mnist_trainset[0][0].size()[2]
# ([1, 28, 28])
N = x_d3 = len(mnist_trainset)
K = 10
x_train = torch.empty((N,x_d0*x_d1*x_d2), device=device)
# (60000, 28*28)

y_train = torch.empty(N, dtype=torch.long)


for i in range(N):
     x_train[i,:] = torch.reshape(mnist_trainset[i][0], (1, x_d0*x_d1*x_d2))
     y_train[i] = mnist_trainset[i][1]
x_train = torch.t(x_train)
y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

# Manipulate test set
N_test = x_d3_test = len(mnist_testset)
x_test = torch.empty((N_test,x_d0*x_d1*x_d2), device=device)
y_test = torch.empty(N_test, dtype=torch.long)
for i in range(N_test):
     x_test[i,:] = torch.reshape(mnist_testset[i][0], (1, x_d0*x_d1*x_d2))
     y_test[i] = mnist_testset[i][1]
x_test = torch.t(x_test)
y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [5]:
#### Reshape X to X-bar
x_trainTensor = torch.reshape(x_train, (x_d1, x_d2, x_d0,-1))
x_testTensor  = torch.reshape(x_test, (x_d1, x_d2, x_d0,-1))
print(x_trainTensor.shape)
print(x_testTensor.shape)

torch.Size([28, 28, 1, 6000])
torch.Size([28, 28, 1, 999])


In [6]:
def inputX_CNN(x_Tensor, filter_size, stride):
    H, W, C, n = x_Tensor.size()
    Hprime = torch.floor(torch.tensor((H-filter_size)/stride))+1
    Hprime = Hprime.to(torch.int)
    Wprime = torch.floor(torch.tensor((W-filter_size)/stride))+1
    Wprime = Wprime.to(torch.int)
    Xtranform = torch.zeros((Hprime * Wprime, filter_size * filter_size * C, n), device=device)
    for i in range(n):
      Data =  x_Tensor[:,:,:,i]
      for hh in range(Hprime):
        for ww in range(Wprime):
          #  print(range(ww * stride, ww * stride + filter_size))
            DataTemp = Data[range(hh * stride, hh * stride + filter_size), :,:]
            DataTemp = DataTemp[:, range(ww * stride, ww * stride + filter_size), :]
            Xtranform[Hprime * hh + ww, :, i] = torch.reshape(DataTemp, (1, filter_size * filter_size * C))
    return Xtranform

In [7]:
### from X-bar to X-bar-bar
filter_size=3
stride = 2
x_trainTS = inputX_CNN(x_trainTensor,filter_size,stride)
x_testTS  = inputX_CNN(x_testTensor,filter_size,stride)

In [8]:

def updateWb_CNN(U, V, W, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)  
    d,N = V.size()
    I = torch.eye(N, device=device)
    U_prime = torch.t(U).reshape(n*Hprime*Wprime,-1)
    Wstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(V),V))+(alpha+tau)*I), rho*torch.mm(torch.t(V),U_prime)+alpha*W+tau*W_tensor2matrix)
    return Wstar

def updateWb_CNNorg(U, V, W, alpha, rho):
    d,N = V.size()
    I = torch.eye(N, device=device)
    #_, col_U = U.size()
    U_prime = torch.t(U).reshape(n*Hprime*Wprime,-1)
    Wstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(V),V))+alpha*I), rho*torch.mm(torch.t(V),U_prime)+alpha*W)
    
    return Wstar

def updateV(U1,U2,W,b,rho,gamma):
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho):
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau):
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [9]:
niter = 2000
rank = 220
tau = 0.2
gamma = 0.5
alpha = 1
rho = 0.5

print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)

S = 32 ### number of filters 2^5
H, W, C, n = x_trainTensor.size()   # n is the same thing as N
Hprime = torch.floor(torch.tensor((H-filter_size)/stride))+1
Hprime = Hprime.to(torch.int)
Wprime = torch.floor(torch.tensor((W-filter_size)/stride))+1
Wprime = Wprime.to(torch.int)



for Out_iter in range(1):
    rank_initial = 700
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)

    d1 =  Hprime * Wprime * S
    d2 =  1024
    d3 =  1024
    d4 =  10


    W1 = 0.01*torch.randn(filter_size * filter_size * C, S, device=device)  
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    std_W2 = torch.sqrt(torch.tensor(1. / d1))
    W2 = torch.empty(d2, d1, device=device).normal_(0, std_W2)
    W2_torch_tensor = W2.reshape((2*Hprime,2*Wprime,4,4,4,4,4,2,2,2)) 
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    b2 = 0*torch.ones(d2, 1, device=device)


    std_W3 = torch.sqrt(torch.tensor(1. / d2))
    W3 = torch.empty(d3, d2, device=device).normal_(0, std_W3)
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) 
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    std_W4 = torch.sqrt(torch.tensor(1. / d3))
    W4 = torch.empty(d4, d3, device=device).normal_(0, std_W4)
    b4 = 0*torch.ones(d4, 1, device=device)

    x_trainTS1 = torch.reshape(x_trainTS, (-1,n))
    x_trainTS1 = torch.t(x_trainTS1)
    x_trainTS1 = torch.reshape(x_trainTS1, (-1, filter_size * filter_size * C))   
    U1prime = torch.matmul(x_trainTS1, W1)
    U1prime = torch.reshape(U1prime, (n,-1))
    U1 = torch.t(U1prime)

    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = U4

    x_testTS1 = torch.reshape(x_testTS, (-1, N_test))
    x_testTS1 = torch.t(x_testTS1)
    x_testTS1 = torch.reshape(x_testTS1, (-1, filter_size * filter_size * C))   


    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

  # update for last layer
        # update V4
        V4 = (y_one_hot + gamma*U4 + alpha*V4)/(1 + gamma + alpha)

        # update U4
        U4 = (gamma*V4 + rho*(torch.mm(W4,V3) + b4.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W4, b4 = updateWb_org(U4,V3,W4,b4,alpha,rho)


  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)


  # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)
        W2_torch_tensor = W2.reshape((2*Hprime,2*Wprime,4,4,4,4,4,2,2,2))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


 # update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        XprimeW = torch.reshape(torch.matmul(x_trainTS1, W1), (n,-1))
        XprimeWtranspose = torch.t(XprimeW)
        # update U1
        U1 = relu_prox(V1,(rho*XprimeWtranspose + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1 = updateWb_CNNorg(U1,x_trainTS1,W1,alpha,rho)






  # prediction for trainning data
        XprimeW = torch.reshape(torch.matmul(x_trainTS1, torch.as_tensor(W1,device=device).reshape((filter_size * filter_size * C, S)).float()), (n,-1))
        XprimeWtranspose = torch.t(XprimeW)
        a1_train = nn.ReLU()(XprimeWtranspose)
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        pred = torch.argmax(torch.addmm(b4.repeat(1, N), W4, a3_train), dim=0)

  #Prediction for test data
        XprimeWtest = torch.reshape(torch.matmul(x_testTS1, torch.as_tensor(W1,device=device).reshape((filter_size * filter_size * C, S)).float()), (N_test,-1))
        XprimeWtesttranspose = torch.t(XprimeWtest)
        a1_test = nn.ReLU()(XprimeWtesttranspose) 
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        pred_test = torch.argmax(torch.addmm(b4.repeat(1, N_test), W4, a3_test), dim=0)


    #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V4,y_one_hot,2),2).cpu().numpy()

        # Eq (5) in paper
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(XprimeWtranspose,U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,U4,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((2*Hprime,2*Wprime,4,4,4,4,4,2,2,2)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \

        # compute training accuracy
        correct_train = pred == y_train
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n',
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k],
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    total_variabels=Sum_of_variables_factors2+Sum_of_variables_factors3

    layer2_CR = Sum_of_variables_factors2/(d1*d2).item()
    layer3_CR = Sum_of_variables_factors3/(d2*d3)
    Compressedlayers_CR = total_variabels/(d1*d2+d2*d3).item()
    Compressedlayers_CR2 = (total_variabels+d3*d4)/(d1*d2+d2*d3+d3*d4).item()



    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(layer2_CR,layer3_CR,Compressedlayers_CR,Compressedlayers_CR2)



rank= 220 tau= 0.2 gamma= 0.5 rho= 0.5 alpha 1


/home/c/cl237/.conda/envs/torch-cuda/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Train on 6000 samples, validate on 999 samples
Repeatition 1 Epoch 1 / 2000 
 - time: 6.645023345947266 - sq_loss: 540.9322509765625 - tot_loss: 740.7799334229902 - acc: 0.3515 - val_acc: 0.34434434434434436
Repeatition 1 Epoch 2 / 2000 
 - time: 6.320704698562622 - sq_loss: 240.41429138183594 - tot_loss: 415.6565715800971 - acc: 0.4846666666666667 - val_acc: 0.47647647647647645
Repeatition 1 Epoch 3 / 2000 
 - time: 6.343242883682251 - sq_loss: 122.63555145263672 - tot_loss: 242.70701812282203 - acc: 0.5593333333333333 - val_acc: 0.5665665665665666
Repeatition 1 Epoch 4 / 2000 
 - time: 6.418163537979126 - sq_loss: 65.06627655029297 - tot_loss: 144.50905776768923 - acc: 0.6256666666666667 - val_acc: 0.6356356356356356
Repeatition 1 Epoch 5 / 2000 
 - time: 6.382887363433838 - sq_loss: 34.95546340942383 - tot_loss: 88.29323547892272 - acc: 0.6766666666666666 - val_acc: 0.6776776776776777
Repeatition 1 Epoch 6 / 2000 
 - time: 6.412253141403198 - sq_loss: 18.877120971679688 - tot_loss: 

Repeatition 1 Epoch 48 / 2000 
 - time: 6.619857311248779 - sq_loss: 0.00028692345949821174 - tot_loss: 0.48510886848089285 - acc: 0.9011666666666667 - val_acc: 0.8968968968968969
Repeatition 1 Epoch 49 / 2000 
 - time: 6.591723442077637 - sq_loss: 0.00026887081912718713 - tot_loss: 0.4737757167604286 - acc: 0.9018333333333334 - val_acc: 0.8978978978978979
Repeatition 1 Epoch 50 / 2000 
 - time: 6.619744062423706 - sq_loss: 0.00025269287289120257 - tot_loss: 0.4630235877528321 - acc: 0.903 - val_acc: 0.8988988988988988
Repeatition 1 Epoch 51 / 2000 
 - time: 6.6387224197387695 - sq_loss: 0.00023819555644877255 - tot_loss: 0.4528067977342289 - acc: 0.9035 - val_acc: 0.8988988988988988
Repeatition 1 Epoch 52 / 2000 
 - time: 6.617673397064209 - sq_loss: 0.00022513615840580314 - tot_loss: 0.44308338593400554 - acc: 0.9036666666666666 - val_acc: 0.8988988988988988
Repeatition 1 Epoch 53 / 2000 
 - time: 6.6297760009765625 - sq_loss: 0.00021335201745387167 - tot_loss: 0.4338159687671577 - a

Repeatition 1 Epoch 95 / 2000 
 - time: 6.712769269943237 - sq_loss: 7.4308984039817e-05 - tot_loss: 0.23719073701213347 - acc: 0.9271666666666667 - val_acc: 0.9179179179179179
Repeatition 1 Epoch 96 / 2000 
 - time: 6.663721323013306 - sq_loss: 7.330306834774092e-05 - tot_loss: 0.23471471483499046 - acc: 0.9275 - val_acc: 0.918918918918919
Repeatition 1 Epoch 97 / 2000 
 - time: 6.63022780418396 - sq_loss: 7.232563075376675e-05 - tot_loss: 0.23229038458521245 - acc: 0.9278333333333333 - val_acc: 0.918918918918919
Repeatition 1 Epoch 98 / 2000 
 - time: 6.6420159339904785 - sq_loss: 7.137871580198407e-05 - tot_loss: 0.22991577400243843 - acc: 0.9281666666666667 - val_acc: 0.918918918918919
Repeatition 1 Epoch 99 / 2000 
 - time: 6.630119562149048 - sq_loss: 7.045074744382873e-05 - tot_loss: 0.22758886617521057 - acc: 0.9288333333333333 - val_acc: 0.918918918918919
Repeatition 1 Epoch 100 / 2000 
 - time: 6.571465969085693 - sq_loss: 6.955027492949739e-05 - tot_loss: 0.2253086419470492 

...

Repeatition 1 Epoch 1794 / 2000 
 - time: 6.635471343994141 - sq_loss: 1.946614929693169e-06 - tot_loss: 0.016605785070669298 - acc: 0.993 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1795 / 2000 
 - time: 6.6650567054748535 - sq_loss: 1.945519670698559e-06 - tot_loss: 0.016597406131131722 - acc: 0.993 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1796 / 2000 
 - time: 6.666805267333984 - sq_loss: 1.9443227756710257e-06 - tot_loss: 0.016589037013000053 - acc: 0.993 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1797 / 2000 
 - time: 6.675849676132202 - sq_loss: 1.942822336786776e-06 - tot_loss: 0.016580674382339566 - acc: 0.993 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1798 / 2000 
 - time: 6.6239073276519775 - sq_loss: 1.941015625561704e-06 - tot_loss: 0.01657232370730526 - acc: 0.993 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1799 / 2000 
 - time: 6.636929988861084 - sq_loss: 1.939152525665122e-06 - tot_loss: 0.016563983394962633 - acc: 0.993 - val_acc: 0.978978

Repeatition 1 Epoch 1843 / 2000 
 - time: 6.606285810470581 - sq_loss: 1.879325509435148e-06 - tot_loss: 0.016204938014243452 - acc: 0.993 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1844 / 2000 
 - time: 6.583258867263794 - sq_loss: 1.8780675645757583e-06 - tot_loss: 0.016196948650235757 - acc: 0.993 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1845 / 2000 
 - time: 6.603668928146362 - sq_loss: 1.8766262428471236e-06 - tot_loss: 0.016188971219412453 - acc: 0.993 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1846 / 2000 
 - time: 6.668275356292725 - sq_loss: 1.875152634056576e-06 - tot_loss: 0.016180997432479673 - acc: 0.993 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1847 / 2000 
 - time: 6.603256464004517 - sq_loss: 1.8740416862783604e-06 - tot_loss: 0.01617303743244065 - acc: 0.993 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1848 / 2000 
 - time: 6.6300740242004395 - sq_loss: 1.8726866528595565e-06 - tot_loss: 0.016165087378567478 - acc: 0.993 - val_acc: 0.9789

Repeatition 1 Epoch 1892 / 2000 
 - time: 5.595665454864502 - sq_loss: 1.8164948869525688e-06 - tot_loss: 0.01582255367443395 - acc: 0.993 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1893 / 2000 
 - time: 5.57732367515564 - sq_loss: 1.8149581819670857e-06 - tot_loss: 0.015814939238430272 - acc: 0.993 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1894 / 2000 
 - time: 5.620131254196167 - sq_loss: 1.8136955759473494e-06 - tot_loss: 0.01580732977606658 - acc: 0.993 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1895 / 2000 
 - time: 5.595816135406494 - sq_loss: 1.8127628891306813e-06 - tot_loss: 0.015799720235986567 - acc: 0.993 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1896 / 2000 
 - time: 5.598578929901123 - sq_loss: 1.8115922557626618e-06 - tot_loss: 0.015792121172808038 - acc: 0.993 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1897 / 2000 
 - time: 5.593965530395508 - sq_loss: 1.8105027947967756e-06 - tot_loss: 0.015784533447435932 - acc: 0.993 - val_acc: 0.97897

Repeatition 1 Epoch 1939 / 2000 
 - time: 5.599061727523804 - sq_loss: 1.7572735941939754e-06 - tot_loss: 0.015471963578556826 - acc: 0.9931666666666666 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1940 / 2000 
 - time: 5.591185569763184 - sq_loss: 1.7556812963448465e-06 - tot_loss: 0.015464674868053408 - acc: 0.9931666666666666 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1941 / 2000 
 - time: 5.602194786071777 - sq_loss: 1.7537087160235387e-06 - tot_loss: 0.01545739565947315 - acc: 0.9931666666666666 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1942 / 2000 
 - time: 5.603945732116699 - sq_loss: 1.7521069821668789e-06 - tot_loss: 0.015450119897013791 - acc: 0.9931666666666666 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1943 / 2000 
 - time: 5.588046073913574 - sq_loss: 1.750634737618384e-06 - tot_loss: 0.015442857551215638 - acc: 0.9931666666666666 - val_acc: 0.978978978978979
Repeatition 1 Epoch 1944 / 2000 
 - time: 5.605424165725708 - sq_loss: 1.7493069890406332e-0

Repeatition 1 Epoch 1985 / 2000 
 - time: 5.627368927001953 - sq_loss: 1.7021239955283818e-06 - tot_loss: 0.015143714382088547 - acc: 0.9933333333333333 - val_acc: 0.97997997997998
Repeatition 1 Epoch 1986 / 2000 
 - time: 5.60288143157959 - sq_loss: 1.7008964050546638e-06 - tot_loss: 0.015136742134723135 - acc: 0.9933333333333333 - val_acc: 0.97997997997998
Repeatition 1 Epoch 1987 / 2000 
 - time: 5.582546949386597 - sq_loss: 1.700475309007743e-06 - tot_loss: 0.01512976472201899 - acc: 0.9933333333333333 - val_acc: 0.97997997997998
Repeatition 1 Epoch 1988 / 2000 
 - time: 5.623313903808594 - sq_loss: 1.7000912748699193e-06 - tot_loss: 0.015122801760685435 - acc: 0.9933333333333333 - val_acc: 0.97997997997998
Repeatition 1 Epoch 1989 / 2000 
 - time: 5.585050582885742 - sq_loss: 1.6987672779578133e-06 - tot_loss: 0.015115846641560893 - acc: 0.9933333333333333 - val_acc: 0.97997997997998
Repeatition 1 Epoch 1990 / 2000 
 - time: 5.586261987686157 - sq_loss: 1.6979431620711694e-06 - to